In [8]:
import sys
sys.path.append("/home/graeme")
sys.path.append("/home/graeme/ElexonDataManager/ElexonDataManager")

In [11]:
import psycopg2
import pandas as pd
import datetime as dt
import gzip
import numpy as np
import pytz
import math
#from ElexonDataManager.BMRA.management.commands import _upload_functions as uf
#from ElexonDataManager.BMRA.management.commands import _download_functions as df
from ElexonDataManager.ElexonDataManager.settings import DATABASES

In [12]:
sys.path

['/home/graeme/ElexonDataManager/notebooks',
 '/home/graeme/anaconda3/lib/python37.zip',
 '/home/graeme/anaconda3/lib/python3.7',
 '/home/graeme/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/home/graeme/anaconda3/lib/python3.7/site-packages',
 '/home/graeme/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/graeme/.ipython',
 '/home/graeme',
 '/home/graeme',
 '/home/graeme/ElexonDataManager',
 '/home/graeme',
 '/home/graeme/ElexonDataManager/ElexonDataManager']

In [23]:
start_date, end_date = dt.date(2017,1,1), dt.date(2019,7,17)
transition_days = [dt.date(x.year, x.month, x.day) for x in pytz.timezone('Europe/London')._utc_transition_times]

In [15]:
P114_df = pd.read_csv('/home/graeme/Downloads/C0421_scotland_df.csv')

In [17]:
bmus = pd.read_csv('/home/graeme/ElexonDataManager/Physical/unit_data/reg_bm_units.csv')

In [18]:
gsp_groups = ['SCOTLAND', 'NORTH of SCOTLAND GSP', 'SOUTH of SCOTLAND GSP']
bmu_ids = list(bmus[bmus['GSP Group Name'].isin(gsp_groups)][~bmus['BM Unit ID'].str.contains('__')]['BM Unit ID'])
bmu_ids

/home/graeme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


['E_ABRTW-1',
 'E_AIRSW-1',
 'E_ASHWW-1',
 'E_ASLVW-1',
 'E_BABAW-1',
 'E_BETHW-1',
 'E_BNWKW-1',
 'E_BRDUW-1',
 'E_BRNLW-1',
 'E_BROXES-1',
 'E_BRYBW-1',
 'E_BTUIW-2',
 'E_CLAC-1',
 'E_CLDRW-1',
 'E_CLFLW-1',
 'E_CNCLW-1',
 'E_CRGTW-1',
 'E_DALSW-1',
 'E_FASN-3',
 'E_FASN-4',
 'E_GDSTW-1',
 'E_GLCHW-1',
 'E_GLNDOD1',
 'E_GLOFW-1',
 'E_HBHDW-1',
 'E_HLGLW-1',
 'E_HLTWW-1',
 'E_HRHLW-1',
 'E_HUNB-11S',
 'E_HYWDW-1',
 'E_KINCW-1',
 'E_MARK-1',
 'E_MINSW-1',
 'E_MOYEW-1',
 'E_TLYMW-1',
 'E_TULWW-1',
 'E_TULWW-2',
 'M_CAS-BEU01',
 'M_CAS-CLU01',
 'M_CAS-CON01',
 'M_CAS-GAR01',
 'M_CAS-KIL01',
 'M_CAS-MOR01',
 'M_SLOY-1',
 'M_SLOY-4',
 'T_ABRBO-1',
 'T_ACHRW-1',
 'T_AFTOW-1',
 'T_ANSUW-1',
 'T_ARCHW-1',
 'T_BDCHW-1',
 'T_BEATD-1',
 'T_BEATO-1',
 'T_BEATO-2',
 'T_BEATO-3',
 'T_BEATO-4',
 'T_BEINW-1',
 'T_BHLAW-1',
 'T_BLKWW-1',
 'T_BLLA-1',
 'T_BLLA-2',
 'T_BPGRD-1',
 'T_BPGRD-2',
 'T_CGTHW-1',
 'T_CLACD-1',
 'T_CLDCW-1',
 'T_CLDNW-1',
 'T_CLDSW-1',
 'T_COCK-1',
 'T_COCK-2',
 'T_COCK-3',
 'T

In [19]:
conn = psycopg2.connect("dbname='ElexonData' user={} host={} password={}".format(DATABASES['default']['USER'],
                                                                                DATABASES['default']['HOST'],
                                                                                DATABASES['default']['PASSWORD']))
cur = conn.cursor()

In [24]:
combined_BAVs = pd.DataFrame(columns = ['sd', 'sp'])
curr_date = start_date
while curr_date <= end_date:
    curr_sp = 1
    if curr_date in transition_days[::2]:
        max_sp = 46
    elif curr_date in transition_days[1::2]:
        max_sp = 50
    else:
        max_sp = 48
    while curr_sp <= max_sp:
        new_data = pd.DataFrame([[curr_date, curr_sp]], columns = ['sd', 'sp'])
        combined_BAVs = combined_BAVs.append(new_data)
        curr_sp += 1
    curr_date += dt.timedelta(days=1)
    
combined_BAVs = combined_BAVs.set_index(['sd', 'sp'])

bav_query = 'SELECT sd, sp, sum(bmra_boav.bv) \
FROM public.bmra_boav \
where bmra_boav.bmu_id=\'%s\' \
group by sd, sp \
order by sd, sp'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(bav_query % bmu_id, conn)
    sql_df = sql_df.set_index(['sd', 'sp'])
    combined_BAVs[bmu_id] = sql_df

combined_BAVs.to_csv('~/scotgov_analysis_190718/BAVs.csv')

In [25]:
combined_OAVs = pd.DataFrame(columns = ['sd', 'sp'])
curr_date = start_date
while curr_date <= end_date:
    curr_sp = 1
    if curr_date in transition_days[::2]:
        max_sp = 46
    elif curr_date in transition_days[1::2]:
        max_sp = 50
    else:
        max_sp = 48
    while curr_sp <= max_sp:
        new_data = pd.DataFrame([[curr_date, curr_sp]], columns = ['sd', 'sp'])
        combined_OAVs = combined_OAVs.append(new_data)
        curr_sp += 1
    curr_date += dt.timedelta(days=1)
    
combined_OAVs = combined_OAVs.set_index(['sd', 'sp'])

oav_query = 'SELECT sd, sp, sum(bmra_boav.ov) \
FROM public.bmra_boav \
where bmra_boav.bmu_id=\'%s\' \
group by sd, sp \
order by sd, sp'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(oav_query % bmu_id, conn)
    sql_df = sql_df.set_index(['sd', 'sp'])
    combined_OAVs[bmu_id] = sql_df

combined_OAVs.to_csv('~/scotgov_analysis_190718/OAVs.csv')

In [26]:
combined_FPNs = pd.DataFrame(columns = ['sd', 'sp'])
curr_date = start_date
while curr_date <= end_date:
    curr_sp = 1
    if curr_date in transition_days[::2]:
        max_sp = 46
    elif curr_date in transition_days[1::2]:
        max_sp = 50
    else:
        max_sp = 48
    while curr_sp <= max_sp:
        new_data = pd.DataFrame([[curr_date, curr_sp]], columns = ['sd', 'sp'])
        combined_FPNs = combined_FPNs.append(new_data)
        curr_sp += 1
    curr_date += dt.timedelta(days=1)
    
combined_FPNs = combined_FPNs.set_index(['sd', 'sp'])
for bmu_id in bmu_ids:
    combined_FPNs[bmu_id] = np.nan

query = 'SELECT sd, sp, bmra_fpnlevel.ts, vp \
FROM public.bmra_fpnlevel \
left join public.bmra_fpn \
on bmra_fpnlevel.fpn_id = bmra_fpn.id \
where bmra_fpn.bmu_id=\'%s\' \
order by sd, sp, bmra_fpnlevel.ts'

for bmu_id in bmu_ids:
    bmu_fpns = []
    print(bmu_id)
    df = pd.read_sql(query % bmu_id, conn)
    prev_SD, prev_SP, prev_VP, prev_TS = None, None, None, None
    total_mwh = 0.0
    for key, values in df.iterrows():
        SD, SP, TS, VP = values['sd'], values['sp'], values['ts'], values['vp']
        if SD==prev_SD and SP==prev_SP: 
            time_diff = (TS - prev_TS).total_seconds() / 3600.0
            total_mwh += (prev_VP * time_diff + (0.5 * (VP - prev_VP) * time_diff))
        elif prev_SD is not None and prev_SP is not None:
            bmu_fpns.append([dt.date(prev_SD.year,prev_SD.month,prev_SD.day),prev_SP,total_mwh])
            total_mwh = 0.0
        prev_SD, prev_SP, prev_VP, prev_TS = SD, SP, VP, TS
    new_data = pd.DataFrame(bmu_fpns, columns=['sd', 'sp', bmu_id])
    new_data = new_data.set_index(['sd', 'sp'])
    combined_FPNs.update(new_data)
    
combined_FPNs.to_csv('~/scotgov_analysis_190718/FPNs.csv')

E_ABRTW-1
E_AIRSW-1
E_ASHWW-1
E_ASLVW-1
E_BABAW-1
E_BETHW-1
E_BNWKW-1
E_BRDUW-1
E_BRNLW-1
E_BROXES-1
E_BRYBW-1
E_BTUIW-2
E_CLAC-1
E_CLDRW-1
E_CLFLW-1
E_CNCLW-1
E_CRGTW-1
E_DALSW-1
E_FASN-3
E_FASN-4
E_GDSTW-1
E_GLCHW-1
E_GLNDOD1
E_GLOFW-1
E_HBHDW-1
E_HLGLW-1
E_HLTWW-1
E_HRHLW-1
E_HUNB-11S
E_HYWDW-1
E_KINCW-1
E_MARK-1
E_MINSW-1
E_MOYEW-1
E_TLYMW-1
E_TULWW-1
E_TULWW-2
M_CAS-BEU01
M_CAS-CLU01
M_CAS-CON01
M_CAS-GAR01
M_CAS-KIL01
M_CAS-MOR01
M_SLOY-1
M_SLOY-4
T_ABRBO-1
T_ACHRW-1
T_AFTOW-1
T_ANSUW-1
T_ARCHW-1
T_BDCHW-1
T_BEATD-1
T_BEATO-1
T_BEATO-2
T_BEATO-3
T_BEATO-4
T_BEINW-1
T_BHLAW-1
T_BLKWW-1
T_BLLA-1
T_BLLA-2
T_BPGRD-1
T_BPGRD-2
T_CGTHW-1
T_CLACD-1
T_CLDCW-1
T_CLDNW-1
T_CLDSW-1
T_COCK-1
T_COCK-2
T_COCK-3
T_COCK-4
T_COCKD-1
T_COUWW-1
T_CRGHD-1
T_CRGHW-1
T_CRMLD-1
T_CRMLW-1
T_CRUA-1
T_CRUA-2
T_CRUA-3
T_CRUA-4
T_CRUAD-1
T_CRYRD-2
T_CRYRW-2
T_CRYRW-3
T_DEAND-1
T_DNLWW-1
T_DOREW-1
T_DOREW-2
T_DRSLW-1
T_DUNGW-1
T_EDINW-1
T_ERRO-1
T_ERRO-2
T_ERRO-3
T_EWHLW-1
T_FALGD-1
T_FALGW-1
T_FARR-1
T_FARR

In [27]:
combined_MELs = pd.DataFrame(columns = ['sd', 'sp'])
curr_date = start_date
while curr_date <= end_date:
    curr_sp = 1
    if curr_date in transition_days[::2]:
        max_sp = 46
    elif curr_date in transition_days[1::2]:
        max_sp = 50
    else:
        max_sp = 48
    while curr_sp <= max_sp:
        new_data = pd.DataFrame([[curr_date, curr_sp]], columns = ['sd', 'sp'])
        combined_MELs = combined_MELs.append(new_data)
        curr_sp += 1
    curr_date += dt.timedelta(days=1)
    
combined_MELs = combined_MELs.set_index(['sd', 'sp'])
for bmu_id in bmu_ids:
    combined_MELs[bmu_id] = np.nan

query = 'SELECT bmra_mel.ts as tsr, sd, sp, bmra_mellevel.ts, ve \
FROM public.bmra_mellevel \
left join public.bmra_mel \
on bmra_mellevel.mel_id = bmra_mel.id \
where bmra_mel.bmu_id=\'%s\' \
order by sd, sp, bmra_mel.ts desc, bmra_mellevel.ts'

for bmu_id in bmu_ids:
    bmu_mels = []
    print(bmu_id)
    df = pd.read_sql(query % bmu_id, conn)
    prev_SD, prev_SP, prev_VP, prev_TS, prev_TSR, last_msg_time = None, None, None, None, None, None
    total_mwh, total_time = 0.0, 0.0
    for key, values in df.iterrows():
        SD, SP, TS, VP, TSR = values['sd'], values['sp'], values['ts'], values['ve'], values['tsr']
        if SD==prev_SD and SP==prev_SP: 
            if TSR==last_msg_time: 
                time_diff = (TS - prev_TS).total_seconds() / 3600.0
                total_mwh += (prev_VP * time_diff + (0.5 * (VP - prev_VP) * time_diff))
                total_time += time_diff
        elif prev_SD is not None and prev_SP is not None:
            bmu_mels.append([dt.date(prev_SD.year,prev_SD.month,prev_SD.day),prev_SP,total_mwh])
            if not math.isclose(0.5, total_time, rel_tol=1e-5):
                print(prev_SD, prev_SP)
                print('total time duration: {:+06.2f}'.format(total_time))
            total_mwh, total_time = 0.0, 0.0
            last_msg_time = values['tsr']
        prev_SD, prev_SP, prev_VP, prev_TS = SD, SP, VP, TS
    new_data = pd.DataFrame(bmu_mels, columns=['sd', 'sp', bmu_id])
    new_data = new_data.set_index(['sd', 'sp'])
    combined_MELs.update(new_data)
    

combined_MELs.to_csv('~/scotgov_analysis_190718/MELs.csv')

E_ABRTW-1
2017-02-20 1
total time duration: +00.00
E_AIRSW-1
2017-03-01 1
total time duration: +00.00
E_ASHWW-1
2016-11-07 34
total time duration: +00.00
E_ASLVW-1
2016-08-01 1
total time duration: +00.00
E_BABAW-1
2014-12-31 3
total time duration: +00.00
E_BETHW-1
2014-12-31 3
total time duration: +00.00
E_BNWKW-1
2017-03-24 1
total time duration: +00.00
E_BRDUW-1
2014-12-31 3
total time duration: +00.00
E_BRNLW-1
2017-04-06 33
total time duration: +00.00
E_BROXES-1
E_BRYBW-1
2014-12-31 3
total time duration: +00.00
E_BTUIW-2
2014-12-31 3
total time duration: +00.00
E_CLAC-1
2014-12-31 3
total time duration: +00.00
2018-03-20 37
total time duration: +00.37
2018-03-22 37
total time duration: +00.37
2018-03-23 33
total time duration: +00.28
2018-03-26 38
total time duration: +00.47
2018-03-27 38
total time duration: +00.48
2018-04-08 16
total time duration: +00.45
2018-04-09 36
total time duration: +00.38
2018-04-10 36
total time duration: +00.43
2018-04-11 16
total time duration: +00.3

T_GLEND-1
T_GLNDO-1
2014-12-31 3
total time duration: +00.00
2018-03-26 24
total time duration: +00.32
T_GLNDOD1
T_GLWSW-1
2016-10-21 1
total time duration: +00.00
T_GNAPW-1
2016-12-13 1
total time duration: +00.00
T_GORDD-1
T_GORDW-1
2014-12-31 3
total time duration: +00.00
T_GRIFW-1
2014-12-31 3
total time duration: +00.00
2018-04-12 30
total time duration: +00.45
T_GRIFW-2
2014-12-31 3
total time duration: +00.00
T_GRIFWD-1
T_GRMO-1
2014-12-31 3
total time duration: +00.00
T_HADHW-1
2014-12-31 3
total time duration: +00.00
T_HRSTW-1
2014-12-31 3
total time duration: +00.00
T_HUNB-7
2014-12-31 3
total time duration: +00.00
2018-03-09 37
total time duration: +00.48
T_HUNB-8
2014-12-31 3
total time duration: +00.00
2018-02-21 43
total time duration: +00.33
2018-02-22 8
total time duration: +00.22
2018-02-22 21
total time duration: +00.22
2018-03-27 8
total time duration: +00.47
2018-04-16 26
total time duration: +00.17
2018-04-16 48
total time duration: +00.22
2018-04-17 4
total time d

In [ ]:
# search BOAV data to figure out what's happening with Peterhead values
filenames = df.get_tibco_daily_filenames(dt.date(2017,3,20), dt.date(2017,3,20))

for filename in filenames:
    file = gzip.open('/home/graeme/ElexonData/Unprocessed/' + filename, 'rb')
    file_content = file.read().decode('utf-8', 'ignore')
    file_rows = [entry for entry in file_content.split('}')]
    count = 0
    for message in file_rows:
        if len(message.strip()) > 0:
            message_dict = uf.message_to_dict(message+'}')
            if message_dict['message_subtype'] == 'BOAV':
                if message_dict['bmu_id'] in ['T_PEHE-1'] and message_dict['SP']==17:
                    print(message_dict)      
            count += 1
    file.close()